# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most. 

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portifolio of these 50 stocks.

# Library imports

In [1]:
from scipy.stats import percentileofscore as score
from scipy import stats
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math

# Importing Our List of Stocks and getting API Token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

data

{'companyName': 'Apple Inc',
 'marketcap': 2361199538844,
 'week52high': 187.35,
 'week52low': 131.86,
 'week52highSplitAdjustOnly': 187.81,
 'week52lowSplitAdjustOnly': 133.6,
 'week52change': 0.013049373074269703,
 'sharesOutstanding': 16470959823,
 'float': 0,
 'avg10Volume': 108915613,
 'avg30Volume': 98012341,
 'day200MovingAvg': 161.15,
 'day50MovingAvg': 162.12,
 'employees': 150025,
 'ttmEPS': 6.25,
 'ttmDividendRate': 0.9078016665124302,
 'dividendYield': 0.006516037604522138,
 'nextDividendDate': '',
 'exDividendDate': '2022-08-02',
 'nextEarningsDate': '2022-10-13',
 'peRatio': 23.95756501343928,
 'beta': 1.253023476293739,
 'maxChangePercent': 56.05809047206917,
 'year5ChangePercent': 3.007257825614793,
 'year2ChangePercent': 0.2648547384904971,
 'year1ChangePercent': 0.003277973824716851,
 'ytdChangePercent': -0.19561384146762,
 'month6ChangePercent': -0.2097513032230892,
 'month3ChangePercent': 0.025407167857558477,
 'month1ChangePercent': -0.1211910341208534,
 'day30Chan

# Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket
notation as in tha projec 1. 

In [4]:
data['year1ChangePercent']

0.003277973824716851

# Executing a batch API Call & Building our Dataframe

In [16]:
#1.Split the list of tickers in sublists.

#2. Using chuncks function
#2.1. Source: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

#3. See batch requests in IEX API documentation

def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [18]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
    #print (symbol_strings[i])
    
my_columns = ['Ticker','Company Name', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string} &types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    print(data['AAPL']['price'])

    for symbol in symbol_string.split(','):
    
        final_dataframe= final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['stats']['companyName'],
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True)
        
final_dataframe

146.57


,Ticker,Company Name,Price,One-Year Price Return,Number of Shares to Buy
0,AAP,Advance Auto Parts Inc,157.550,-0.241819,N/A
1,AAPL,Apple Inc,146.570,0.003214,N/A
2,ABBV,Abbvie Inc,138.090,0.367462,N/A
3,ABC,Amerisource Bergen Corp.,140.880,0.127828,N/A
4,ABMD,Abiomed Inc.,260.280,-0.250174,N/A
...,...,...,...,...,...
95,CMA,"Comerica, Inc.",71.700,-0.096947,N/A
96,CMCSA,Comcast Corp,30.125,-0.448516,N/A
97,CME,CME Group Inc,185.760,-0.074978,N/A
98,CMG,Chipotle Mexican Grill,1589.410,-0.169389,N/A


# Removing Low-Momentum Stocks

In [8]:
# The parameter inplace = True overwrites the dataframe with the ascending filter selection.
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe.reset_index(inplace = True)
final_dataframe[:20]


,index,Ticker,Company Name,Price,One-Year Price Return,Number of Shares to Buy
0,86,CF,CF Industries Holdings Inc,100.610,0.747313,N/A
1,40,APA,APA Corporation,34.819,0.690271,N/A
2,91,CI,Cigna Corp.,283.500,0.398997,N/A
3,75,CAH,"Cardinal Health, Inc.",67.220,0.377283,N/A
4,2,ABBV,Abbvie Inc,138.090,0.367462,N/A
5,9,ADM,Archer Daniels Midland Co.,83.500,0.363010,N/A
6,35,ANET,Arista Networks Inc,117.840,0.339670,N/A
7,37,ANTM,Anthem Inc,491.620,0.286952,N/A
8,52,AZO,Autozone Inc.,2162.780,0.252075,N/A
9,21,ALB,Albemarle Corp.,265.990,0.230815,N/A


# Calculating the Number of Shares to Buy (based on Price)

In [9]:
portifolio_size = 100000
val = float(portifolio_size)

position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])

final_dataframe

,index,Ticker,Company Name,Price,One-Year Price Return,Number of Shares to Buy
0,86,CF,CF Industries Holdings Inc,100.610,0.747313,9
1,40,APA,APA Corporation,34.819,0.690271,28
2,91,CI,Cigna Corp.,283.500,0.398997,3
3,75,CAH,"Cardinal Health, Inc.",67.220,0.377283,14
4,2,ABBV,Abbvie Inc,138.090,0.367462,7
...,...,...,...,...,...,...
95,43,APTV,Aptiv PLC,80.110,-0.484863,12
96,7,ADBE,Adobe Inc,287.310,-0.521966,3
97,90,CHTR,Charter Communications Inc.,314.018,-0.596921,3
98,82,CCL,Carnival Corp.,7.521,-0.667521,132


# Building a better (and more realistic) momentum strategy

Real world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks: 

High-quality momentum stocks show "slow and steady" outperformance over long periods of time. 
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (shuch as an FDA approval for a biotechnology company). 

To identify high-quality momentum , we're going to build a strategy that selects stocks from the highest percentiles of:

1. 1-month price returns
2. 3-month price returns
3. 6-month price returns
4. 1-year price returns
5. 2-year price returns
6. 5-year price returns

Ps: hqm = high-quality-momentum


In [10]:
hqm_columns =[
    'Ticker', 
    'Company Name',
    'Price', 
    'Number of Shares to Buy',
    'Five-Year Price Return',
    'Five-Year Return Percentile',
    'Two-Year Price Return',
    'Two-Year Return Percentile',    
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [11]:
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string} &types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
    
        hqm_dataframe= hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['stats']['companyName'],
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year5ChangePercent'],
            'N/A',
            data[symbol]['stats']['year2ChangePercent'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index = hqm_columns),
        ignore_index = True)
        
hqm_dataframe

,Ticker,Company Name,Price,Number of Shares to Buy,Five-Year Price Return,Five-Year Return Percentile,Two-Year Price Return,Two-Year Return Percentile,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAP,Advance Auto Parts Inc,164.570,N/A,0.726305,N/A,0.101160,N/A,-0.245944,N/A,-0.250386,N/A,-0.092667,N/A,-0.056736,N/A,N/A
1,AAPL,Apple Inc,145.494,N/A,3.029183,N/A,0.273670,N/A,0.003246,N/A,-0.203540,N/A,0.025149,N/A,-0.118680,N/A,N/A
2,ABBV,Abbvie Inc,138.100,N/A,1.024281,N/A,0.807991,N/A,0.378635,N/A,-0.108608,N/A,-0.066134,N/A,0.052311,N/A,N/A
3,ABC,Amerisource Bergen Corp.,137.810,N/A,0.815493,N/A,0.458542,N/A,0.126544,N/A,-0.119457,N/A,-0.050847,N/A,-0.077993,N/A,N/A
4,ABMD,Abiomed Inc.,253.670,N/A,0.494251,N/A,-0.086473,N/A,-0.245807,N/A,-0.250395,N/A,-0.029570,N/A,-0.031620,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CMA,"Comerica, Inc.",74.150,N/A,0.147730,N/A,1.107982,N/A,-0.096482,N/A,-0.238674,N/A,-0.041594,N/A,-0.102914,N/A,N/A
96,CMCSA,Comcast Corp,30.535,N/A,-0.137968,N/A,-0.329147,N/A,-0.463449,N/A,-0.367284,N/A,-0.235322,N/A,-0.163529,N/A,N/A
97,CME,CME Group Inc,178.530,N/A,0.540528,N/A,0.135502,N/A,-0.072543,N/A,-0.255004,N/A,-0.130325,N/A,-0.082084,N/A,N/A
98,CMG,Chipotle Mexican Grill,1576.570,N/A,4.038474,N/A,0.227428,N/A,-0.170075,N/A,-0.030977,N/A,0.203990,N/A,-0.061039,N/A,N/A


In [12]:
time_periods = [
    'Five-Year',
    'Two-Year',
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index: 
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        price_col = 'Price'
        percentile_col = f'{time_period} Return Percentile'
        #hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

        try:
             hqm_dataframe.loc[row, percentile_col] = round(score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col]))
        except:
            hqm_dataframe.loc[row, percentile_col] = 0
        
hqm_dataframe

,Ticker,Company Name,Price,Number of Shares to Buy,Five-Year Price Return,Five-Year Return Percentile,Two-Year Price Return,Two-Year Return Percentile,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAP,Advance Auto Parts Inc,164.570,N/A,0.726305,63,0.101160,48,-0.245944,29,-0.250386,37,-0.092667,28,-0.056736,82,N/A
1,AAPL,Apple Inc,145.494,N/A,3.029183,97,0.273670,63,0.003246,75,-0.203540,48,0.025149,76,-0.118680,36,N/A
2,ABBV,Abbvie Inc,138.100,N/A,1.024281,82,0.807991,91,0.378635,97,-0.108608,68,-0.066134,41,0.052311,98,N/A
3,ABC,Amerisource Bergen Corp.,137.810,N/A,0.815493,71,0.458542,79,0.126544,84,-0.119457,64,-0.050847,48,-0.077993,67,N/A
4,ABMD,Abiomed Inc.,253.670,N/A,0.494251,47,-0.086473,22,-0.245807,30,-0.250395,36,-0.029570,62,-0.031620,89,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CMA,"Comerica, Inc.",74.150,N/A,0.147730,27,1.107982,97,-0.096482,56,-0.238674,43,-0.041594,53,-0.102914,51,N/A
96,CMCSA,Comcast Corp,30.535,N/A,-0.137968,13,-0.329147,7,-0.463449,6,-0.367284,11,-0.235322,6,-0.163529,12,N/A
97,CME,CME Group Inc,178.530,N/A,0.540528,50,0.135502,49,-0.072543,61,-0.255004,32,-0.130325,19,-0.082084,62,N/A
98,CMG,Chipotle Mexican Grill,1576.570,N/A,4.038474,99,0.227428,59,-0.170075,43,-0.030977,84,0.203990,96,-0.061039,79,N/A


# Calculating the HQM Score

We'll now calculate our HQM Score, which is the high-quality momentum score
that we'll use to filter for stocks in this investing strategy. 

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores
that we calculated in the last section.

To calculate arithmetic mean, we will use the mean funcion from Python built-in
statistics module.

In [13]:
from statistics import mean

for row in hqm_dataframe.index: 

    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    
    try:
        hqm_dataframe.loc[row,'HQM Score'] = round(mean(momentum_percentiles))
    except:
        hqm_dataframe.loc[row,'HQM Score'] = 0

hqm_dataframe.head(20)

,Ticker,Company Name,Price,Number of Shares to Buy,Five-Year Price Return,Five-Year Return Percentile,Two-Year Price Return,Two-Year Return Percentile,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAP,Advance Auto Parts Inc,164.570,N/A,0.726305,63,0.101160,48,-0.245944,29,-0.250386,37,-0.092667,28,-0.056736,82,48
1,AAPL,Apple Inc,145.494,N/A,3.029183,97,0.273670,63,0.003246,75,-0.203540,48,0.025149,76,-0.118680,36,66
2,ABBV,Abbvie Inc,138.100,N/A,1.024281,82,0.807991,91,0.378635,97,-0.108608,68,-0.066134,41,0.052311,98,80
3,ABC,Amerisource Bergen Corp.,137.810,N/A,0.815493,71,0.458542,79,0.126544,84,-0.119457,64,-0.050847,48,-0.077993,67,69
4,ABMD,Abiomed Inc.,253.670,N/A,0.494251,47,-0.086473,22,-0.245807,30,-0.250395,36,-0.029570,62,-0.031620,89,48
5,ABT,Abbott Laboratories,100.290,N/A,1.012068,81,-0.036565,31,-0.165035,44,-0.181518,52,-0.099473,22,-0.035906,88,53
6,ACN,Accenture plc,268.760,N/A,1.071913,83,0.189548,55,-0.193488,40,-0.246196,38,-0.077255,38,-0.128233,29,47
7,ADBE,Adobe Inc,288.970,N/A,0.872951,77,-0.437403,2,-0.539202,4,-0.423246,7,-0.255439,5,-0.266521,2,16
8,ADI,Analog Devices Inc.,145.480,N/A,0.849520,74,0.263089,62,-0.149810,47,-0.158298,56,-0.031575,60,-0.087098,58,60
9,ADM,Archer Daniels Midland Co.,84.400,N/A,1.217890,85,0.849479,93,0.360571,95,-0.074831,75,0.049717,83,-0.112246,42,79


# Selecting the 50 best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the
HQM Score column and dropping all but the top 50 entries.

In [14]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe.reset_index(inplace = True , drop = True)

hqm_dataframe_Rankbest = hqm_dataframe[:20]
hqm_dataframe_Rankbest[['Ticker', 'Company Name','HQM Score','Price']].head(20)


,Ticker,Company Name,HQM Score,Price
0,AZO,Autozone Inc.,91,2239.430
1,ALB,Albemarle Corp.,91,268.840
2,ANTM,Anthem Inc,86,501.910
3,CDNS,"Cadence Design Systems, Inc.",86,169.655
4,CI,Cigna Corp.,86,279.000
5,ADP,Automatic Data Processing Inc.,85,236.460
6,AJG,Arthur J. Gallagher & Co.,85,175.206
7,ANET,Arista Networks Inc,85,115.160
8,CF,CF Industries Holdings Inc,83,100.820
9,CAH,"Cardinal Health, Inc.",83,67.080


# Calculating the number of shares to buy

We'll use the portifolio_input function that we created earlier to accept our portifolio size. Then we'll use similar logic in
for loop to calculate the number of shares to buy for each stock in our investment universe.

In [15]:
portifolio_input = 100000
position_size = float(portifolio_input)/50

for i in hqm_dataframe_Rankbest.index:
    hqm_dataframe_Rankbest.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe_Rankbest.loc[i, 'Price'])
    
hqm_dataframe_Rankbest


c:\users\feoxp7\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Company Name,Price,Number of Shares to Buy,Five-Year Price Return,Five-Year Return Percentile,Two-Year Price Return,Two-Year Return Percentile,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AZO,Autozone Inc.,2239.430,0,2.686050,96,0.872735,95,0.250150,92,0.039549,95,0.009119,73,-0.003897,95,91
1,ALB,Albemarle Corp.,268.840,7,1.084982,84,2.124131,99,0.221712,90,0.197943,98,0.256111,98,-0.069797,75,91
2,ANTM,Anthem Inc,501.910,3,1.753076,92,0.943289,96,0.287758,93,0.056339,96,0.006381,71,-0.071466,71,86
3,CDNS,"Cadence Design Systems, Inc.",169.655,11,3.264711,98,0.567655,81,0.082268,81,-0.007820,87,0.092262,92,-0.058229,80,86
4,CI,Cigna Corp.,279.000,7,0.547310,51,0.759577,90,0.401500,98,0.164119,97,0.051755,84,-0.023594,93,86
5,ADP,Automatic Data Processing Inc.,236.460,8,1.322706,87,0.728183,88,0.166113,88,0.022387,93,0.081694,91,-0.083329,61,85
6,AJG,Arthur J. Gallagher & Co.,175.206,11,2.067284,94,0.694475,85,0.155424,87,-0.003546,89,0.051959,85,-0.072877,69,85
7,ANET,Arista Networks Inc,115.160,17,1.458793,89,1.210831,98,0.340056,94,-0.190657,49,0.216205,97,-0.058055,81,85
8,CF,CF Industries Holdings Inc,100.820,19,2.169406,95,2.426881,100,0.781493,100,-0.020102,86,0.134322,94,-0.149310,21,83
9,CAH,"Cardinal Health, Inc.",67.080,29,0.213523,30,0.579609,82,0.375895,96,0.201148,99,0.277580,99,-0.028580,90,83
